<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 KB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141231 sha256=c874acf02d74ea29299699bdabdbced6f78dca83b12c12a8ce8451d648ba146d
  Stored in directory: /root/.cache/pip/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
Successfully built antlr4-python3-runtime


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.1 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
#import torch
import os
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor
import librosa
import fairseq
import torch
import pandas as pd

RuntimeError: ignored

In [ ]:
def segment_audio(audio_file):
    audio_input, sample_rate = librosa.load(audio_file)
    audio_segment = sample_rate * 3
    segments = []
    if len(audio_input) >= audio_segment * 2:
      check = True
      while check:
          segments.append(audio_input[:audio_segment])
          audio_input = audio_input[audio_segment:]
          if len(audio_input) < audio_segment:
              check = False
    else:
      segments.append(audio_input[:audio_segment])
    return segments, sample_rate

# return first 3 second segment in audio file
def three_second(audio_file):
    audio_input, sample_rate = librosa.load(audio_file)
    audio_segment = sample_rate * 3

    segment = audio_input[:audio_segment]
    return segment, sample_rate


def read_event(tsv_path):
    event = pd.read_csv("sub-A2014_task-auditory_events.tsv", sep = '\t')
    event = event[event['type'] == 'Sound']

    batch["onset"], batch["sample"], batch["audio_file"] = event['onset'], event['sample'], event['audio_file']
    
    return batch


def prepare_dataset(batch, feature_extractor):
    for audio in audio_file_directory:
        if audio in batch["audio_file"]:
            segment, sample_rate = three_second(audio)
            #segment, sample_rate = segment_audio(audio)
            
            feature_extractor = prepare_feature_extractor(sample_rate)
            batch["input_values"] = feature_extractor(segment, return_tensors="pt").input_values

    return batch

def prepare_model(model_path):
    model = Wav2Vec2ForCTC.from_pretrained(model_path)

    return model

def prepare_feature_extractor(sampling_rate):
    # check that all files have the correct sampling rate
    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=sampling_rate, padding_value=0.0, do_normalize=True, return_attention_mask=True)

    return feature_extractor

In [ ]:
model_path = "facebook/wav2vec2-large-xlsr-53"
model = prepare_model(model_path)

NameError: ignored

In [ ]:
import re
import numpy as np
import torch
import os
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor
import librosa
import fairseq
import torch
import pandas as pd


model_path = "facebook/wav2vec2-large-xlsr-53"
audio_path = "/content/audio"

model = prepare_model(model_path)
files = os.listdir(audio_path)

# for wav_file in files:
#     count = 0
#     name = re.search('(\d+)\.', wav_file).group().replace('.', '_')

#     audio = os.path.join(audio_path, wav_file)

#     audio_input, sample_rate = segment_audio(audio)
#     feature_extractor = prepare_feature_extractor(sample_rate)

#     if len(audio_input) > 1:
#         for audio in audio_input:
#             input_values = feature_extractor(audio, return_tensors="pt").input_values
#             logits = model(input_values).logits

#             output = logits.detach().numpy()
#             np.save(f'{name}{count}.npy', output)

#             count += 1
#     else:
#         input_values = feature_extractor(audio_input, return_tensors="pt").input_values
#         logits = model(input_values).logits

#         output = logits.detach().numpy()
#         np.save(f'{name}.npy', output)

# Only first three second segements
for wav_file in files:
    print(wav_file)
    name = re.search('(\d+)\.', wav_file).group().replace('.', '')

    audio = os.path.join(audio_path, wav_file)

    audio_input, sample_rate = three_second(audio)
    feature_extractor = prepare_feature_extractor(sample_rate)

    input_values = feature_extractor(audio_input, return_tensors="pt").input_values
    logits = model(input_values).logits

    output = logits.detach().numpy()
    np.save(os.path.join('/content/Embedding/MyDrive/Embedding', f'{name}.npy'), output)
